In [ ]:
# example of training a gan on mnist
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot

# define the standalone discriminator worker
def descriminator_fun(in_shape=(28,28,1)):
	worker = Sequential()
	worker.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
	worker.add(LeakyReLU(alpha=0.2))
	worker.add(Dropout(0.4))
	worker.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
	worker.add(LeakyReLU(alpha=0.2))
	worker.add(Dropout(0.4))
	worker.add(Flatten())
	worker.add(Dense(1, activation='sigmoid'))
	# compile worker
	optimizer = Adam(lr=0.0002, beta_1=0.5)
	worker.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return worker

# define the standalone generator worker
def generative_fun(latent_internal_dimensions):
	worker = Sequential()
	# foundation for 7x7 image
	n_nodes = 128 * 7 * 7
	worker.add(Dense(n_nodes, input_dim=latent_internal_dimensions))
	worker.add(LeakyReLU(alpha=0.2))
	worker.add(Reshape((7, 7, 128)))
	# upsample to 14x14
	worker.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	worker.add(LeakyReLU(alpha=0.2))
	# upsample to 28x28
	worker.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	worker.add(LeakyReLU(alpha=0.2))
	worker.add(Conv2D(1, (7,7), activation='sigmoid', padding='same'))
	return worker

# define the combined generator and discriminator worker, for updating the generator
def adversial_fun(generative_model, discriminator_model):
	# make weights in the discriminator not trainable
	discriminator_model.trainable = False
	# connect them
	worker = Sequential()
	# add generator
	worker.add(generative_model)
	# add the discriminator
	worker.add(discriminator_model)
	# compile worker
	optimizer = Adam(lr=0.0002, beta_1=0.5)
	worker.compile(loss='binary_crossentropy', optimizer=optimizer)
	return worker

# load and prepare mnist training images
def loadMNISTdata():
	# load mnist data
	(trainX, _), (_, _) = load_data()
	# expand to 3d, e.g. add channels dimension
	X = expand_dims(trainX, axis=-1)
	# convert from unsigned ints to floats
	X = X.astype('float32')
	# scale from [0,255] to [0,1]
	X = X / 255.0
	return X

# select real samples
def sampling_and_testing_data(data, n_samples):
	# choose random instances
	ix = randint(0, data.shape[0], n_samples)
	# retrieve selected images
	X = data[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, 1))
	return X, y

# generate points in latent space as input for the generator
def gen_latent_internal(latent_internal_dimensions, n_samples):
	# generate points in the latent space
	x_input = randn(latent_internal_dimensions * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_internal_dimensions)
	return x_input

# use the generator to generate n fake examples, with class labels
def fake_samples_fun(generative_model, latent_internal_dimensions, n_samples):
	# generate points in latent space
	x_input = gen_latent_internal(latent_internal_dimensions, n_samples)
	# predict outputs
	X = generative_model.predict(x_input)
	# create 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y

# create and save a plot of generated images (reversed grayscale)
def gen_plot(examples, epoch, n=10):
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
	# save plot to file
	filename = 'generated_plot_e%03d.png' % (epoch+1)
	pyplot.savefig(filename)
	pyplot.close()

# evaluate the discriminator, plot generated images, save generator worker
def summarize_performance(epoch, generative_model, discriminator_model, data, latent_internal_dimensions, n_samples=100):
	# prepare real samples
	X_real, y_real = sampling_and_testing_data(data, n_samples)
	# evaluate discriminator on real examples
	_, acc_real = discriminator_model.evaluate(X_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = fake_samples_fun(generative_model, latent_internal_dimensions, n_samples)
	# evaluate discriminator on fake examples
	_, acc_fake = discriminator_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	# save plot
	gen_plot(x_fake, epoch)
	# save the generator worker tile file
	filename = 'generator_model_%03d.h5' % (epoch + 1)
	generative_model.save(filename)

# train the generator and discriminator
def train(generative_model, discriminator_model, adversial_model, data, latent_internal_dimensions, n_epochs=100, n_batch=256):
	bat_per_epo = int(data.shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			X_real, y_real = sampling_and_testing_data(data, half_batch)
			# generate 'fake' examples
			X_fake, y_fake = fake_samples_fun(generative_model, latent_internal_dimensions, half_batch)
			# create training set for the discriminator
			X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
			# update discriminator worker weights
			d_loss, _ = discriminator_model.train_on_batch(X, y)
			# prepare points in latent space as input for the generator
			X_gan = gen_latent_internal(latent_internal_dimensions, n_batch)
			# create inverted labels for the fake samples
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = adversial_model.train_on_batch(X_gan, y_gan)
			# summarize loss on this batch
			print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))
		# evaluate the worker performance, sometimes
		if (i+1) % 10 == 0:
			summarize_performance(i, generative_model, discriminator_model, data, latent_internal_dimensions)

# size of the latent space
latent_internal_dimensions = 100
# create the discriminator
discriminator_model = descriminator_fun()
# create the generator
generative_model = generative_fun(latent_internal_dimensions)
# create the gan
adversial_model = adversial_fun(generative_model, discriminator_model)
# load image data
data = loadMNISTdata()
# train worker
train(generative_model, discriminator_model, adversial_model, data, latent_internal_dimensions)

/usr/local/lib/python3.9/dist-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


4/4 [==============================] - 4s 668ms/step
>1, 1/234, d=0.721, g=0.625
4/4 [==============================] - 1s 187ms/step
>1, 2/234, d=0.708, g=0.644
4/4 [==============================] - 1s 312ms/step
>1, 3/234, d=0.705, g=0.661
4/4 [==============================] - 1s 189ms/step
>1, 4/234, d=0.691, g=0.684
4/4 [==============================] - 2s 358ms/step
>1, 5/234, d=0.684, g=0.706
4/4 [==============================] - 1s 196ms/step
>1, 6/234, d=0.675, g=0.729
4/4 [==============================] - 1s 188ms/step
>1, 7/234, d=0.661, g=0.753
4/4 [==============================] - 1s 189ms/step
>1, 8/234, d=0.657, g=0.768
4/4 [==============================] - 1s 184ms/step
>1, 9/234, d=0.645, g=0.793
4/4 [==============================] - 1s 202ms/step
>1, 10/234, d=0.641, g=0.813
4/4 [==============================] - 1s 192ms/step
>1, 11/234, d=0.628, g=0.833
4/4 [==============================] - 1s 183ms/step
>1, 12/234, d=0.628, g=0.846
4/4 [====================